# Ernesto Antonio Reyes Ramírez

# Aprendizaje Máquina

# Tarea: redes recurrentes

La base de datos que me tocó fue la del punto 5 que es sobre el índice de precios al consumidor en la Unión Europea. Descargué de la página de Eurostats los datos sobre el HICP (índice de precios al consumidor) sobre la comida en Francía. Los datos son mensuales, así que haremos las predicciones mensualmente.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#Herramientas para el modelo neuronal
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, GRU

# Procesamiento de los datos

In [2]:
#Cargamos los archivos de google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#Leemos la base de datos que deacargamos de EuroStats
datos = pd.read_csv('/content/gdrive/MyDrive/DB_HICP/HICP_FR_FOOD.csv')

In [4]:
#Mostramos los datos
datos

,DATAFLOW,LAST UPDATE,freq,unit,coicop,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,1996-01,72.19,NaN
1,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,1996-02,72.35,NaN
2,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,1996-03,73.01,NaN
3,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,1996-04,72.97,NaN
4,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,1996-05,73.73,NaN
...,...,...,...,...,...,...,...,...,...
322,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,2022-11,124.21,NaN
323,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,2022-12,124.79,NaN
324,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,2023-01,127.08,NaN
325,ESTAT:PRC_HICP_MIDX(1.0),02/05/23 11:00:00,M,I15,CP011,FR,2023-02,129.43,NaN


In [5]:
#Limpiamos los datos
df = datos.drop(["DATAFLOW","LAST UPDATE", "freq","unit","coicop", "geo","OBS_FLAG"], axis=1)

In [6]:
df

,TIME_PERIOD,OBS_VALUE
0,1996-01,72.19
1,1996-02,72.35
2,1996-03,73.01
3,1996-04,72.97
4,1996-05,73.73
...,...,...
322,2022-11,124.21
323,2022-12,124.79
324,2023-01,127.08
325,2023-02,129.43


In [7]:
# Procesamos los datos y etiquetas. Recordamos que haremos una predicción de dados m pasos, calcularemos los siguientes n
m = 20  # número de pasos anteriores
n = 5   # número de pasos futuros

X, y = [], []

for i in range(m, len(df)):
    X.append(df['OBS_VALUE'].values[i-m:i])
    y.append(df['OBS_VALUE'].values[i:i+n])

In [8]:
X = X[:-4]
y = y[:-4]

In [9]:
X = np.array(X)
y = np.array(y)

### Train, val, test

In [10]:
X_train = X[:243]
y_train = y[:243]

X_val = X[243:273]
y_val = y[243:273]

X_test = X[273:303]
y_test = y[273:303]

# Modelos

Modelo SimpleRNN

In [11]:
model1 = Sequential()
model1.add(SimpleRNN(32, input_shape=(m, 1)))
model1.add(Dense(5, activation='sigmoid'))

In [12]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 32)                1088      
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 1,253
Trainable params: 1,253
Non-trainable params: 0
_________________________________________________________________


In [13]:
model1.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['MeanSquaredError'])

In [14]:
import time
tic = time.time()
history = model1.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=(X_val, y_val),
                    verbose=2)
print('Tiempo de entrenamiento:', time.time()-tic)

Epoch 1/100
16/16 - 6s - loss: 8175.9316 - mean_squared_error: 8175.9316 - val_loss: 11175.6699 - val_mean_squared_error: 11175.6699 - 6s/epoch - 361ms/step
Epoch 2/100
16/16 - 0s - loss: 8146.4209 - mean_squared_error: 8146.4209 - val_loss: 11140.6318 - val_mean_squared_error: 11140.6318 - 246ms/epoch - 15ms/step
Epoch 3/100
16/16 - 0s - loss: 8113.7300 - mean_squared_error: 8113.7300 - val_loss: 11097.4004 - val_mean_squared_error: 11097.4004 - 235ms/epoch - 15ms/step
Epoch 4/100
16/16 - 0s - loss: 8087.2339 - mean_squared_error: 8087.2339 - val_loss: 11083.8184 - val_mean_squared_error: 11083.8184 - 238ms/epoch - 15ms/step
Epoch 5/100
16/16 - 0s - loss: 8080.0977 - mean_squared_error: 8080.0977 - val_loss: 11077.9990 - val_mean_squared_error: 11077.9990 - 259ms/epoch - 16ms/step
Epoch 6/100
16/16 - 0s - loss: 8076.0024 - mean_squared_error: 8076.0024 - val_loss: 11074.2188 - val_mean_squared_error: 11074.2188 - 236ms/epoch - 15ms/step
Epoch 7/100
16/16 - 0s - loss: 8073.2651 - mean_

In [15]:
# Evaluar el rendimiento en el conjunto de prueba
loss, accuracy = model1.evaluate(X_test, y_test)

print("Loss en el conjunto de prueba:", loss)
print("Precisión en el conjunto de prueba:", accuracy)

1/1 [==============================] - 0s 37ms/step - loss: 12659.3770 - mean_squared_error: 12659.3770
Loss en el conjunto de prueba: 12659.376953125
Precisión en el conjunto de prueba: 12659.376953125


# LSTM

In [16]:
model2 = Sequential()
model2.add(LSTM(32, input_shape=(m, 1)))
model2.add(Dense(5, activation='sigmoid'))

In [17]:
model2.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['MeanSquaredError'])

In [18]:
tic = time.time()
history = model2.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=(X_val, y_val),
                    verbose=2)
print('Tiempo de entrenamiento:', time.time()-tic)

Epoch 1/100
16/16 - 3s - loss: 8161.9932 - mean_squared_error: 8161.9932 - val_loss: 11169.8662 - val_mean_squared_error: 11169.8662 - 3s/epoch - 217ms/step
Epoch 2/100
16/16 - 0s - loss: 8150.7280 - mean_squared_error: 8150.7280 - val_loss: 11157.0039 - val_mean_squared_error: 11157.0039 - 87ms/epoch - 5ms/step
Epoch 3/100
16/16 - 0s - loss: 8138.8916 - mean_squared_error: 8138.8916 - val_loss: 11142.0586 - val_mean_squared_error: 11142.0586 - 90ms/epoch - 6ms/step
Epoch 4/100
16/16 - 0s - loss: 8124.8618 - mean_squared_error: 8124.8618 - val_loss: 11123.8623 - val_mean_squared_error: 11123.8623 - 86ms/epoch - 5ms/step
Epoch 5/100
16/16 - 0s - loss: 8110.4009 - mean_squared_error: 8110.4009 - val_loss: 11109.1748 - val_mean_squared_error: 11109.1748 - 88ms/epoch - 5ms/step
Epoch 6/100
16/16 - 0s - loss: 8098.0547 - mean_squared_error: 8098.0547 - val_loss: 11094.1396 - val_mean_squared_error: 11094.1396 - 101ms/epoch - 6ms/step
Epoch 7/100
16/16 - 0s - loss: 8087.0991 - mean_squared_e

In [19]:
# Evaluar el rendimiento en el conjunto de prueba
loss, accuracy = model2.evaluate(X_test, y_test)

print("Loss en el conjunto de prueba:", loss)
print("Precisión en el conjunto de prueba:", accuracy)

1/1 [==============================] - 0s 33ms/step - loss: 12659.2285 - mean_squared_error: 12659.2285
Loss en el conjunto de prueba: 12659.228515625
Precisión en el conjunto de prueba: 12659.228515625


# GatedRNN

In [20]:
model3 = Sequential()
model3.add(GRU(32, input_shape=(m, 1)))
model3.add(Dense(5, activation='sigmoid'))

In [21]:
model3.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['MeanSquaredError'])

In [22]:
tic = time.time()
history = model3.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=(X_val, y_val),
                    verbose=2)
print('Tiempo de entrenamiento:', time.time()-tic)

Epoch 1/100
16/16 - 4s - loss: 8128.1304 - mean_squared_error: 8128.1304 - val_loss: 11128.5518 - val_mean_squared_error: 11128.5518 - 4s/epoch - 224ms/step
Epoch 2/100
16/16 - 0s - loss: 8114.1030 - mean_squared_error: 8114.1030 - val_loss: 11113.0176 - val_mean_squared_error: 11113.0176 - 90ms/epoch - 6ms/step
Epoch 3/100
16/16 - 0s - loss: 8100.2324 - mean_squared_error: 8100.2324 - val_loss: 11096.2617 - val_mean_squared_error: 11096.2617 - 96ms/epoch - 6ms/step
Epoch 4/100
16/16 - 0s - loss: 8083.9404 - mean_squared_error: 8083.9404 - val_loss: 11079.3232 - val_mean_squared_error: 11079.3232 - 89ms/epoch - 6ms/step
Epoch 5/100
16/16 - 0s - loss: 8075.4805 - mean_squared_error: 8075.4805 - val_loss: 11073.1660 - val_mean_squared_error: 11073.1660 - 90ms/epoch - 6ms/step
Epoch 6/100
16/16 - 0s - loss: 8072.3569 - mean_squared_error: 8072.3569 - val_loss: 11070.5713 - val_mean_squared_error: 11070.5713 - 100ms/epoch - 6ms/step
Epoch 7/100
16/16 - 0s - loss: 8070.4941 - mean_squared_e

In [23]:
# Evaluar el rendimiento en el conjunto de prueba
loss, accuracy = model3.evaluate(X_test, y_test)

print("Loss en el conjunto de prueba:", loss)
print("Precisión en el conjunto de prueba:", accuracy)

1/1 [==============================] - 0s 26ms/step - loss: 12659.3633 - mean_squared_error: 12659.3633
Loss en el conjunto de prueba: 12659.36328125
Precisión en el conjunto de prueba: 12659.36328125


# Conclusión

La verdad que estuve probando muchas configuraciones y buscando en internet pero no encontré una solución a que el entrenamiento de tan malos resultados. No sé si es por los datos o por como los manejé. Creo que los procesé correctamente para ser series de tiempo y querer predecir 5 al futuro con 20 del pasado.